In [8]:
# Import some numerical math and plotting packages
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

# Import the MesaProfile package
from MesaProfile import MesaProfile

# Import the OrderedDict package for working with MesaProfile outputs
from collections import OrderedDict

# Read the input MESA profile                                                                                                                                                                                      
mesa = MesaProfile('profile64.data')
mstar = mesa.getStar()    #gives dic of fields inside profile

In [9]:
# List the data fields available in the profile
for k in mstar.keys():  
    print(k)

zone
logT
logRho
logP
logR
luminosity
logL
velocity
entropy
conv_mixing_type
csound
v_div_csound
v_div_r
eta
mu
logdq
dq_ratio
q
radius
temperature
tau
logtau
pressure
pgas_div_ptotal
logPgas
grada
cp
logS
gam
free_e
chiRho
chiT
abar
zbar
z2bar
ye
log_opacity
eps_nuc
non_nuc_neu
eps_grav
mlt_mixing_length
log_D_mix
log_conv_vel
conv_vel_div_csound
log_mlt_D_mix
pressure_scale_height
gradT
gradr
dlnd_dt
dlnT_dt
mass
mmid
logxq
h1
he3
he4
c12
n14
o16
ne20
mg24
si28
pp
cno
tri_alfa
burn_c
burn_n
burn_o
burn_ne
burn_na
burn_mg
burn_si
burn_s
burn_ar
burn_ca
burn_ti
burn_cr
burn_fe
c12_c12
c12_o16
o16_o16
pnhe4
photo
other
extra_heat
gradr_sub_grada
brunt_N2
brunt_B
brunt_nonB
sign_brunt_N2
lamb_S2
lamb_S
log_brunt_nu
log_lamb_Sl2
log_lamb_Sl3
brunt_N_div_r_integral
k_r_integral
brunt_N2_sub_omega2
sl2_sub_omega2
logQ
radiuscm
model_number
num_zones
initial_mass
initial_z
star_age
time_step
Teff
photosphere_L
photosphere_r
center_eta
center_h1
center_he3
center_he4
center_c12
center_n14
cen

In [10]:
print(mstar)

OrderedDict([('zone', array([4670, 4669, 4668, ...,    3,    2,    1])), ('logT', array([ 8.90681299,  8.90681197,  8.90681078, ...,  7.0292963 ,
        7.02681864,  7.02341436])), ('logRho', array([ 9.66511783,  9.6651162 ,  9.66511426, ..., -0.68603295,
       -0.69350332, -0.70371653])), ('logP', array([ 27.57290096,  27.57289878,  27.57289618, ...,  14.2294201 ,
        14.21947902,  14.20585976])), ('logR', array([-5.85545746, -5.75511366, -5.65476959, ..., -2.57834948,
       -2.57833857, -2.57832741])), ('luminosity', array([ 155752.91738267,  311495.59796466,  622956.90815246, ...,
         15294.33145995,   15294.51791051,   15294.70863798])), ('logL', array([ 5.19243619,  5.49345191,  5.79445801, ...,  4.1845305 ,
        4.18453579,  4.18454121])), ('velocity', array([  1.01312906e+00,   1.27646394e+00,   1.60824638e+00, ...,
         5.80601186e+04,   7.31220318e+04,  -9.54019023e-02])), ('entropy', array([  0.59640509,   0.596405  ,   0.59640491, ...,  11.78939432,
      

In [11]:
## Show all plots
plt.show()


# Map abundances to set of C12, O16, Ne20, Ne22 for flash!
fmap = {'c12':np.array([]),'o16':np.array([]),'ne20':np.array([]),'ne22':np.array([])}
## Maintain constant C12 abundance
fmap['c12'] = mstar['c12']
## Determine Ne22 abundance from model Ye
fmap['ne22'] = np.maximum(22.0*(0.5-mstar['ye']), 0.0)
## Normalization requires Ne20 + O16 abundances = xother
xother = 1.0 - fmap['c12'] - fmap['ne22']
## Ne20/O16 ratio remains constant
rneo = mstar['ne20']/mstar['o16']
## Use rneo and xother constraints to find Ne20 and O16 abundances
fmap['o16'] = xother/(rneo+1.0)
fmap['ne20'] = rneo*xother/(rneo+1.0)

# Write new abundances to output file (one line per zone)
## Note we assume normalization so o16 makes up the difference!
## Note mesa's radius units are Rsun so I convert to cm for output
## Format is:
## l.1: # radius         dens           temp        c12          ne20          ne22
## l.2: [Number of lines to follow]
## l.3: data according to header
fout = open('profile64.output.data','w')
fout.write('# radius         dens           pres           temp        c12          ne20          ne22\n')
numpts = len(fmap['c12']) # Can use something else if, e.g. you mass-average
fout.write(str(numpts) + '\n')

def esf(x):
	return '{0:0.15e}'.format(x)

for i in range(0,numpts):
	fout.write(esf(mstar['radiuscm'][i]) + '  ' +
			esf(10.0**mstar['logRho'][i]) + '  ' + 
			esf(mstar['pressure'][i]) + '  ' + 
			esf(mstar['temperature'][i]) + '  ' +
			esf(fmap['c12'][i]) + '  ' +
			esf(fmap['ne20'][i]) + '  ' +
			esf(fmap['ne22'][i]) + '\n')

# All done, save file!
fout.close()